In [0]:
spark.conf.set('fs.azure.account.key.storemysynapsedata.dfs.core.windows.net',"YFmWW1rUyjQ3oVlKUT9nt3IIL/YChFOHOYFhAzsxj4Qwx0+34lQsTP6aJFpEM5YnERglw+7nrsmF+AStya1S6g==")

In [0]:
from pyspark.sql.functions import explode, col, sum, array, collect_list

df = spark.read.json("abfss://raw@storemysynapsedata.dfs.core.windows.net/orders.json", multiLine=True)
df = df.withColumn('items_new', explode('items'))
df2 = df.select(
    col("customer.email").alias("email"),
    col("customer.name").alias("customer_name"),
    col("delivery.address").alias("address"),
    col("delivery.status").alias("delivery_status"),
    col("items_new.item").alias("item"),
    col("items_new.price").alias("price"),
    col("items_new.qty").alias("qty"),
    col("order_id").alias("order_id")
)
# df2.display()

df_total_revenue = df2.groupBy('email').agg(sum(col('price') * col('qty')).alias('total_revenue'))
df_total_quantity = df2.groupBy('item').agg(sum(col('qty')).alias('total_quantity'))
df_total_revenue_per_item=df2.groupBy('item').agg(sum(col('price')*col('qty')).alias('total_revenue_per_item'))
# df5=df2.withColumn('items_purchased', array('item')).withColumn('total_revenue', col('price')*col('qty')) --nooo
df_customer_summary=df2.groupBy('email','customer_name').agg(sum(col('price')*col('qty')).alias("total_revenue"),collect_list('item').alias('items_purchased'))


df_customer_summary.display()

In [0]:
data = [
    (1, "Tiger tger Tigre"),
    (2, "Tigers Tigress Tiger Tiger Tiger"),
    (3, "Tiger tger Tigre Tigerees"),
    (4, "Tigers Tiger tiger"),
    (5, "Tigre Tiger Tigress tiger")
]

schema=['id','text']

df=spark.createDataFrame(data,schema)
df.show()

In [0]:
from pyspark.sql.functions import split, explode, lower

df1=df.withColumn('Tiger_array', split(df.text," "))
df2 = df1.withColumn('text_into_rows', explode('Tiger_array'))
df3=df2.filter(lower(df2.text_into_rows) == 'tiger')

df4=df3.groupBy('id','text').count()
df4.display()

In [0]:
from pyspark.sql.functions import avg
from pyspark.sql.window import Window

data = [
    (1, "Alice", "HR", 50000),
    (2, "Bob", "HR", 60000),
    (3, "Charlie", "IT", 80000),
    (4, "David", "IT", 75000),
    (5, "Eve", "Finance", 90000),
    (6, "Frank", "Finance", 85000),
    (7, "Grace", "IT", 95000),
]
columns = ["emp_id", "name", "department", "salary"]

df=spark.createDataFrame(data, columns)
# df=df.groupBy('department').agg(avg('salary').alias('avg_salary'))
# df2
window_spec=Window.partitionBy('department')
df2=df.withColumn('avg_salary',avg('salary').over(window_spec))
df2.filter(df2.salary>df2.avg_salary).display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Spark Playground").getOrCreate()

data = [
    (1, "Alice", "HR", 50000, "2024-01-01"),
    (1, "Alice", "HR", 52000, "2024-03-01"),
    (2, "Bob", "IT", 70000, "2024-02-10"),
    (2, "Bob", "IT", 70000, "2024-02-10"),
    (3, "Charlie", "Finance", 90000, "2024-05-05"),
    (3, "Charlie", "Finance", 88000, "2024-01-10"),
]

columns = ["emp_id", "name", "department", "salary", "updated_at"]
df = spark.createDataFrame(data, columns)
df.display()


In [0]:
df=df.withColumn('updated_at', F.col('updated_at').cast('date'))
# df=df.withColumn('updated_at', F.to_date(F.col('updated_at')))

Window1=Window.partitionBy('emp_id').orderBy(F.col('updated_at').desc())

df1=df.withColumn('rank',F.row_number().over(Window1))
df2=df1.filter(df1.rank==1).drop('rank')
df2.display()

In [0]:
lines = [
    "Hello world!, my world is coding world",
    "Hello PySpark, excuse me Hello",
    "PySpark is fun. and fun is writing pyspark code",
    "Hello!, Hello world again"
]
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("WordCountDF").getOrCreate()

# Create DataFrame
df = spark.createDataFrame(lines, "string").toDF("line")

# Your code starts here
# 1. Convert to lowercase, split lines into words, explode, filter empty strings
# 2. Group by word and count
# 3. Sort by count descending


In [0]:
from pyspark.sql.functions import split, explode, lower, col

df1=df.withColumn('new', split(lower(df.line),' '))
df2=df1.withColumn('new2',explode(df1.new))
df3=df2.groupBy('new2').count()
df3=df3.sort(col('count').desc()) # sort, orderBy does same work. note: To do desc() we need to specify inside sort() or orderBy()
df3.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("GroupByAggregateList").getOrCreate()

data = [
    (1, "Alice", "HR", "Recruitment"),
    (2, "Bob", "HR", "Training"),
    (3, "Charlie", "IT", "Migration"),
    (4, "David", "IT", "Security"),
    (5, "Frank", "Finance", "Budgeting"),
    (5, "Frank", "Finance", "Reporting"),
    (7, "Grace", "IT", "Reporting"),
    (7, "Grace", "IT", "Automation")
]

columns = ["emp_id", "name", "department", "project"]

df = spark.createDataFrame(data, columns)

# Your task: group by department and aggregate projects into a list
# df_result = ?

df.display()


In [0]:
df=df.groupBy('emp_id','name','department').agg(F.array_sort(collect_list('project')).alias('project_list'))
df.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("AdvancedGroupByArrays").getOrCreate()

data = [
    (1, "Alice", "Laptop", 1200),
    (1, "Alice", "Mouse", 25),
    (1, "Alice", "Keyboard", 75),
    (2, "Bob", "Monitor", 300),
    (2, "Bob", "Mouse", 25),
    (3, "Charlie", "Laptop", 1300),
    (3, "Charlie", "Mouse", 30),
    (3, "Charlie", "Desk", 200)
]

columns = ["customer_id", "customer_name", "product", "amount"]

df = spark.createDataFrame(data, columns)

# Your task:
# 1. Group by customer_id (or customer_name)
# 2. Aggregate product names into a list
# 3. Aggregate amounts into a list
# 4. Compute total_spent
# 5. Sort product list alphabetically


In [0]:
# NO SORTING BECAUSE SORTING MAY REARRANGE THE PRODUCTS->AMOUNTS CORRESPONDENCE
# (because sorting the product list separately could break the correspondence between products and amounts)

df1=df.groupBy('customer_id','customer_name').agg(F.collect_list('product').alias('products'), F.collect_list('amount').alias('amounts'), F.sum('amount').alias('total_spent'))

df1.display()

customer_id,customer_name,products,amounts,total_spent
1,Alice,"List(Laptop, Mouse, Keyboard)","List(1200, 25, 75)",1300
2,Bob,"List(Monitor, Mouse)","List(300, 25)",325
3,Charlie,"List(Laptop, Mouse, Desk)","List(1300, 30, 200)",1530


In [0]:
# SAFE SORTING

df1=df.groupBy('customer_id','customer_name').agg(F.array_sort(F.collect_list(F.struct('product','amount'))).alias('product_amounts'), F.sum('amount').alias('total_spent'))
df2=df1.select('customer_id','customer_name',"product_amounts.product","product_amounts.amount",'total_spent')
df2.display()

customer_id,customer_name,product,amount,total_spent
1,Alice,"List(Keyboard, Laptop, Mouse)","List(75, 1200, 25)",1300
2,Bob,"List(Monitor, Mouse)","List(300, 25)",325
3,Charlie,"List(Desk, Laptop, Mouse)","List(200, 1300, 30)",1530


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("MonthlyTransactionSummary").getOrCreate()

data = [
    (1, "Alice", "2025-01-15", 200),
    (2, "Bob", "2025-01-20", 300),
    (3, "Charlie", "2025-02-05", 150),
    (4, "David", "2025-02-18", 400),
    (5, "Eve", "2025-03-01", 500),
    (6, "Frank", "2025-03-15", 250),
    (7, "Grace", "2025-03-20", 100)
]

columns = ["transaction_id", "customer_name", "transaction_date", "amount"]

df = spark.createDataFrame(data, columns)

# Ensure transaction_date is a DateType
# df = df.withColumn("transaction_date", F.to_date("transaction_date", "yyyy-MM-dd"))

# Your task:
# 1. Extract month and year
# 2. Group by year and month
# 3. Compute total transactions, total amount, and average amount


In [0]:
# Ensure transaction_date is a DateType
df = df.withColumn("transaction_date", F.to_date("transaction_date", "yyyy-MM-dd"))
df1=df.withColumn('YEAR', F.year(df.transaction_date)).withColumn('MONTH', F.month(df.transaction_date)) #.drop(df.transaction_date)
df2=df1.groupBy('year','month').agg(F.count('*').alias('total_transactions'), F.sum('amount').alias('total_amount'), F.avg('amount').alias('average_amount'))

# df2=df1.groupBy('year','month').agg({"*":"count", "amount": "sum", "amount":"avg"}) # KEYS ARE REPEATING IT CONSIDERS LATEST KEY WHICH MISSES total_sales ("amount": "sum")


df2.display()

year,month,total_transactions,total_amount,average_amount
2025,1,2,500,250.0
2025,2,2,550,275.0
2025,3,3,850,283.3333333333333
